In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar 

--2022-03-06 07:00:04--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.72MB/s    in 0.2s    

2022-03-06 07:00:05 (5.72 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon_book_review").getOrCreate()

In [4]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz'
spark.sparkContext.addFile(url)

schema = StructType([ 
    StructField("marketplace",StringType(),True), 
    StructField("customer_id",IntegerType(),True), 
    StructField("review_id",StringType(),True), 
    StructField("product_id", IntegerType(), True),
    StructField("product_parent", IntegerType(), True),
    StructField("product_title", StringType(), True),
    StructField("product_category", StringType(), True),
    StructField("star_rating", IntegerType(), True),
    StructField('helpful_vote', IntegerType(), True),
    StructField('total_votes', IntegerType(), True),
    StructField('vine', StringType(), True),
    StructField('verified_purchase', StringType(), True),
    StructField('review_headline', StringType(), True),
    StructField('review_body', StringType(), True),
    StructField('review_date', StringType(), True),
  ])

df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_00.tsv.gz"), sep="\t", header=True, schema=schema, timestampFormat="yyyy/MM/dd HH:mm:ss")

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_vote|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO| 439873800|      84656342|There Was an Old ...|           Books|          5|           0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|        

In [6]:
# total records (number of rows): 10319090 rows (this is huge man)
df.count()

10319090

In [7]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_vote: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
# review_id table
review_id_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_df = review_id_df.dropna()
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RJOVP071AVAJO|   25933450| 439873800|      84656342| 2015-08-31|
|R1ORGBETCDW3AI|    1801372|1623953553|     729938122| 2015-08-31|
|R2NYB6C3R8LVN6|   14005703|1604600527|     800572372| 2015-08-31|
|R13U5PBJI1H94K|   36205738| 399170863|     559876774| 2015-08-31|
|R1H8UVH990F8VE|   44121167|1517007240|     299984591| 2015-08-31|
|R2MC0N30WZMRQ5|   16519255| 671728725|     821650353| 2015-08-31|
| R2NO2HXK16Y4J|   49361350|1111349533|     419457767| 2015-08-31|
| RFBMLBWQOZ1UM|   37536260| 679413359|      58983679| 2015-08-31|
| RHE1FEWV3KTQH|   23614950| 983990662|     222024699| 2015-08-31|
|R2G519UREHRO8M|   49735028| 664254969|     248307276| 2015-08-31|
| RPBKCEIN3UISD|   39348901|1570913722|       9340721| 2015-08-31|
|R2DOB8482VGJ8W|    5782091| 345313860|     630892090| 2015-08

In [9]:
# products table
products_df = df.select(['product_id', 'product_title'])
products_df = products_df.dropna()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
| 439873800|There Was an Old ...|
|1623953553|      I Saw a Friend|
|1604600527|Stars 'N Strips F...|
| 399170863|            The Liar|
|1517007240|Devil in the Deta...|
| 671728725|Knowing When to S...|
|1111349533|The American Pageant|
| 679413359|The Aeneid (Every...|
| 983990662|The YMCA of Middl...|
| 664254969|Presbyterian Cree...|
|1570913722|Hello Ocean/Hola mar|
| 345313860|The Vampire Lesta...|
| 670025593|The Book of Life ...|
|1616892285|The Bike Deconstr...|
|1512172227|The One-Day Weeke...|
|1938067126|Crimes of the Edu...|
|1940595223|High School Runne...|
| 321955358|Microbiology with...|
|1607747308|The Life-Changing...|
| 133083608|Pearson Reviews &...|
+----------+--------------------+
only showing top 20 rows



In [10]:
# customers table
# customers_df = df.select(['customer_id', ''])
customer_df = df.groupBy('customer_id').count()
customer_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   16747249|    4|
|     108460|    1|
|   11720077|   31|
|     437936|    1|
|     128131|    1|
|   39647402|    1|
|   13730111|    8|
|   43783459|    1|
|   12493167|  111|
|   40325486|   92|
|   44618558|    4|
|   17956020|    1|
|   10093406|    3|
|   40430762|    1|
|     134924|    1|
|   52427934|    1|
|   12879980|   15|
|   27680937|    7|
|   16186289|    1|
|   20326078|    1|
+-----------+-----+
only showing top 20 rows



In [11]:
# vine table
vine_df = df.select(['review_id', 'star_rating', 'helpful_vote', 'total_votes', 'vine'])
vine_df = vine_df.dropna()
vine_df.show()


+--------------+-----------+------------+-----------+----+
|     review_id|star_rating|helpful_vote|total_votes|vine|
+--------------+-----------+------------+-----------+----+
| RJOVP071AVAJO|          5|           0|          0|   N|
|R1ORGBETCDW3AI|          5|           0|          0|   N|
| R7TNRFQAOUTX5|          5|           0|          0|   N|
|R2GANXKDIFZ6OI|          5|           0|          0|   N|
|R2NYB6C3R8LVN6|          5|           2|          2|   N|
|R13U5PBJI1H94K|          2|           1|          1|   N|
|R1H8UVH990F8VE|          5|           2|          2|   N|
|R2MC0N30WZMRQ5|          5|           0|          0|   N|
| R2NO2HXK16Y4J|          5|           0|          0|   N|
|R245YIAVJK82ZL|          5|           0|          0|   N|
| RFBMLBWQOZ1UM|          5|           0|          0|   N|
| RHE1FEWV3KTQH|          5|           0|          0|   N|
|R2BB9U7PHO7ETO|          5|           0|          1|   N|
|R28XJJP754GYXO|          4|           0|          0|   

In [12]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/classroom_database"
config = {"user":"root", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}

In [15]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table_1', mode=mode, properties=config)

In [18]:
products_df.write.jdbc(url=jdbc_url, table="products_1", mode=mode, properties=config)

In [17]:
customer_df.write.jdbc(url=jdbc_url, table="customers_1", mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table_1", mode=mode, properties=config)